In [ ]:
! pip install lppls

In [ ]:
# Write PSY bubble prediction algorithm in python

import numpy as np
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from lppls import lppls, data_loader, lppls_cmaes


def PSY(y, swindow0=None, IC=0, adflag=0):

    t = len(y)

    if swindow0 is None:
        swindow0 = int(np.floor(t * (0.01 + 1.8 / np.sqrt(t))))

    bsadfs = np.empty(t)
    bsadfs[:] = np.nan

    for r2 in tqdm(range(swindow0, t)):
        rwadft = np.empty(r2 - swindow0 + 1)
        rwadft[:] = -999
        for r1 in range(0, r2 - swindow0 + 1):
            rwadft[r1] = adfuller(y[r1:r2+1], maxlag=adflag, autolag=None)[0]

        bsadfs[r2] = np.max(rwadft)

    bsadf = np.hstack((np.zeros(swindow0), bsadfs[swindow0:t]))

    return bsadf

In [ ]:
price_data_df = pd.read_csv('data/30m_BTC_USD.csv')
# price_data_df.drop(columns=['Unnamed: 0'], inplace=True)
# Take first 100 rows
start, length = 22000, 1000

price_data_df = price_data_df.iloc[start:start+length]
price_data_df['PSY_3h'] = PSY(price_data_df['close'].values, swindow0=2*3, IC=0, adflag=0)
price_data_df['PSY_6h'] = PSY(price_data_df['close'].values, swindow0=2*6, IC=0, adflag=0)
price_data_df['PSY_1d'] = PSY(price_data_df['close'].values, swindow0=2*24, IC=0, adflag=0)
price_data_df['PSY_3d'] = PSY(price_data_df['close'].values, swindow0=2*24*3, IC=0, adflag=0)
price_data_df['PSY_7d'] = PSY(price_data_df['close'].values, swindow0=2*24*7, IC=0, adflag=0)
# price_data_df['PSY_int'] = price_data_df['PSY'].apply(lambda x: int(x))

In [ ]:
psy95 = np.percentile(price_data_df['PSY'], 95)
psy95

In [ ]:
price_data_df['PSY'] = price_data_df['PSY'].apply(lambda x: 1 if x >= psy95 else 0)

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=price_data_df['timestamp'], y=price_data_df['close'], name='Price'),
              row=1, col=1)
fig.add_trace(go.Scatter(x=price_data_df['timestamp'], y=price_data_df['PSY'], name='PSY'),
              row=2, col=1)
# fig.add_trace(go.Scatter(x=price_data_df['timestamp'], y=price_data_df['PSY_int'], name='PSY Integer'),
#               row=2, col=1)
fig.show()


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=price_data_df['timestamp'], y=price_data_df['close'], name='Price'), secondary_y=False)
fig.add_trace(go.Scatter(x=price_data_df['timestamp'], y=price_data_df['PSY'], name='PSY'), secondary_y=True)
# fig.add_trace(go.Scatter(x=price_data_df['timestamp'], y=price_data_df['PSY_int'], name='PSY Integer'), secondary_y=True)
fig.show()